## Bariogenesis

In [1]:
import math
import scipy.integrate
import numpy as np
import pandas as pd
import time

from matplotlib import pyplot as plt

from mpmath import *
mp.pretty= True
mp.dps=20

import random

In [2]:
### Definitions of general functions ###
### We've set $\kappa=1$ 

infinite = np.inf

def my_heaviside(x1,x2):
    
    if x1 < 0:
        x = 0.
        
    elif x1 == 0.:
        x = x2        
    
    else:
        x=1.
    
    return x

def mycosh(x):
    ### Our own definition of coh(x) to avoid evaluating np.cosh(x) at |x|>150 since this will have precission issues 
    ###
        
    if abs(x)< 150.0:
        return np.cosh(x)
    else:
        return 1.e60

def Scpv(xpm, vpzsq, pzovom, th, lam, so, Lw, mo, Tn, vw, z):
    # This is the definition of the CP-violation source from Cline's work
    
    mchi1=abs(so*(1.0 + np.tanh(z/Lw))/2.0) #error 1/2
    mchisq=((mo+lam*np.cos(th)*mchi1))**2.0 + (lam*np.sin(th)*mchi1)**2.0 #revisar
    
    #vz = p/np.sqrt(p**2.0 + mchisq)
    scpv = -(2.0*vw*np.pi/(lam*Tn**3.0)*pzovom)*(mo*so*(-2.0+mycosh(2.0*z/Lw))*np.sin(th)/(Lw**3*mycosh(z/Lw)**4.0))
    
    # From Andres' code, I took the idea of renormalizing this function 
    if np.isnan(scpv):
        Scpv_renormalizado = 1.e50
    else:
        Scpv_renormalizado = scpv
        
    return Scpv_renormalizado
    
def xiL(vw2,xpm, vpzsq, pzovom, th, lam, so, Lw, mo, Tn, vw, Mzp, z):
    # \xi_{L} \def 3(n_{\chi_L} - n_{\chi^c_L})/T_c^3
    
    gfcn= lambda z: 1.0/(vw*np.sqrt(1.0+8.0*vpzsq/(3.0*vw2)))*(my_heaviside(z,1.0)*np.exp(-z*Tn*(3.0+np.sqrt(9.0+24.0*vpzsq/vw2))*vw*lam**2.0/(8.0*np.pi*vpzsq))
            +my_heaviside(-z,0.0)*np.exp(-z*Tn*(3.0-np.sqrt(9.0+24.0*vpzsq/vw2))*vw*lam**2.0/(8.0*np.pi*vpzsq)))
    inttemp = lambda zo: Scpv(xpm, vpzsq, pzovom, th, lam, so, Lw, mo, Tn, vw, zo)*gfcn(z-zo)
    return scipy.integrate.quad(inttemp,-30.,30.,limit=40)[0]

def DelnBeq(th, lam, so, Lw, mo, Tn, vw, Mzp, qS, gp, z):
    ###  thermal equilibrium asymmetry
    
    vw2=vw**2
    xpm=mo/Tn 
    vpzsq= (3.0*xpm+2.0)/(xpm**2.0+3.0*xpm+2.0)
    pzovom=((1.0-xpm)*np.exp(-xpm)+xpm**2.0*scipy.special.erf(xpm))/(4.0*(xpm*Tn)**2.0*scipy.special.kn(2, xpm))
    
    inttemp = lambda z1: xiL(vw2, xpm, vpzsq,  pzovom, th, lam, so, Lw, mo, Tn, vw, Mzp, z1)*np.exp(-Mzp*abs(z-z1))
    return gp*qS*Tn**2/3.0*gp/(Mzp)*(1.0/3.0*qS*Tn**3)*scipy.integrate.quad(inttemp,-30.,30.,limit=40)[0]



parameters

In [3]:
'''
### For now, we fix all parameters except gp and MZp

tht=math.pi/3.0 
lamt=0.3
sot=100.0
Lwt=5.0/100.0
mot=100 #mchi[i]
#pt=10.0 
Tnt=100.0
vsqt=0.7
vwt=0.1
qS=5.0 #charge of Phi
'''

load file

In [3]:
yd = pd.read_csv('good-points.csv')
del[yd['Unnamed: 0']]

In [4]:
yd = yd.reset_index()
del[yd['index']]

In [5]:
yd[['MZp','g1p']][:3]

,MZp,g1p
0,331.683687,0.106056
1,766.251487,0.142847
2,726.393759,0.139681


In [6]:
for i in range(0,3):
    Mzpt=yd['MZp'][i]
    gpt=yd['g1p'][i]
    
    print(Mzpt,gpt)

331.683687 0.106055722082
766.251487 0.142847270439
726.393759 0.139680891753


fixed parameters

In [7]:
qS=5.0 #charge of Phi

In [11]:
### Loop for scan of MZp and gp

t1= time.time()

yt=[]

for i in range(0,1):

    Mzpt=yd['MZp'][i]
    gpt=yd['g1p'][i]
    mot=yd['mchi'][i]
    
    for j in range(0,10):
        ## random parameters
        tht = np.random.uniform(-np.pi/2.,np.pi/2.)
        lamt= np.exp(np.random.uniform(np.log(1.*10**(-2)),np.log(10**(0))))
        sot= np.random.uniform(100.,500.)
        Tnt= np.random.uniform(100.,500.)
        Lwt= np.random.uniform(1./Tnt,10./Tnt)
        vwt= np.random.uniform(0.05,0.5)

        xt=[]
        xt2=[]
        for zt in np.arange(0,20.,.36): 
            ### This for loop creates an interpolation of \Delta n_{B eq} vs z. To be used in the numerical integral that
            ### follows

            delnbt=DelnBeq(tht, lamt, sot, Lwt, mot, Tnt, vwt, Mzpt, qS, gpt,zt)
            #print(zt, ' ',delnbt)
            xt.append(delnbt)
            xt2.append(zt)

        dnbfcn = lambda z: np.interp(z,xt2,xt)
        inttemp = lambda z: dnbfcn(z)*np.exp(-z*120.0*(gpt**2.0/(4.0*math.pi))**5.0*Tnt/vwt)
        delnbt= scipy.integrate.quad(inttemp,0.0,20.0,limit=40)[0]*120.0*(gpt**2.0/(4.0*math.pi))**5.0*Tnt/vwt

        #entdens=(2.0*math.pi**2.0)*100.0*(125.0)**3.0/45.0 ### This is s ~ Tc^3 Tc =125.0
        #print('MZ\'=',Mzpt, ' gp=',gpt,delnbt/entdens)
        yt.append([Mzpt,gpt,delnbt,mot,tht,lamt,sot,Tnt,Lwt,vwt])

yt=np.asarray(yt)

ydt=pd.DataFrame(yt,columns=['MZp','gp','Deltan_B','mot','tht','lamt','sot','Tnt','Lwt','vwt'])

ydt.to_csv('Mzvsgp3.csv')

t2= time.time()

/usr/lib/python3/dist-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


In [12]:
print(t2-t1)

66.26538395881653


In [13]:
ydt

,MZp,gp,Deltan_B,mot,tht,lamt,sot,Tnt,Lwt,vwt
0,331.683687,0.106056,2.985798e-14,409.149868,-0.269305,0.316942,320.824443,149.476168,0.053449,0.061176
1,331.683687,0.106056,-4.190716e-14,409.149868,1.075149,0.272951,130.201734,149.671196,0.017920,0.151726
2,331.683687,0.106056,1.675218e-15,409.149868,-1.026507,0.014549,493.746924,359.067218,0.025517,0.343335


In [10]:
ydt

,MZp,gp,Deltan_B
0,331.683687,0.106056,5.369271e-23


In [10]:
ydt

,MZp,gp,Deltan_B
0,331.683687,0.106056,7.811186e-14


In [61]:
ydt

,MZp,gp,Deltan_B
0,331.683687,0.106056,5.734053e-14


In [57]:
ydt

,MZp,gp,Deltan_B
0,331.683687,0.106056,-7.870809e-14


In [47]:
ydt

,MZp,gp,Deltan_B
0,21.115896,0.000579,4.377222e-37
1,4.033246,0.000194,1.133835e-42
2,1650.346290,0.067420,-1.598465e-36
3,159.098225,0.045725,-1.058748e-17
4,2327.030710,0.041038,1.442633e-82


In [13]:
ydt

,MZp,gp,Deltan_B
0,10.0,0.01,4.213581e-22
